In [1]:
from openai import OpenAI
client = OpenAI()

### Defining the assistant

"tools" argument 
    - it is a dictionary specifying which functions can be used
    - parameters used in the function
    - description of the functions and of the parameters used.

Specifying the tools here, will allow GPT to understand which functions have to be called and which parameters

In [2]:
assistant = client.beta.assistants.create(
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-4-turbo",
  tools=[
    {
      "type": "function",
      "function": {
        "name": "get_current_temperature",
        "description": "Get the current temperature for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": ["Celsius", "Fahrenheit"],
              "description": "The temperature unit to use. Infer this from the user's location."
            }
          },
          "required": ["location", "unit"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "get_rain_probability",
        "description": "Get the probability of rain for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            }
          },
          "required": ["location"]
        }
      }
    }
  ]
)

### Creating a thread

Give instructions to the GPT

In [3]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="What's the weather in London today and the likelihood it'll rain?",
)

### Run

In [20]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

### Analyse output

So now the API was called and the model run.

Given we passed some tools, the GPT will try to call the right functions - if necessary.

If tools are called then **run.status == requires_action**, this will tell us tools need to be used.

GPT won't allow the code to run auotmatically and we have to pipe the GPT output into the code to run the functions we want to be called.

This is an example and the functions simply returns 57 degrees, and 6% chance of rain

In [23]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)
 
# Define the list to store tool outputs
tool_outputs = []

requires_action


**And these are the functions that GPT wants to call and the relative parameters**

In [29]:
run.required_action.submit_tool_outputs.tool_calls

[RequiredActionFunctionToolCall(id='call_Ix8b8lQ4JA0W5zooEi5m8hmj', function=Function(arguments='{"location": "London, UK", "unit": "Celsius"}', name='get_current_temperature'), type='function'),
 RequiredActionFunctionToolCall(id='call_avs1mEO80smIHaHl2DYuqWBf', function=Function(arguments='{"location": "London, UK"}', name='get_rain_probability'), type='function')]

### Tool output

We create a list **tool_output**, where we sget the function output and the **tool_call_id**.

the tool id is necessary because we will feed back to the GPT the functions output, but we have to let the GPT knows to which function the output is referring to.

The GPT will then be able to provide the correct answer, using the tools you gave it.

In [ ]:
# Loop through each tool in the required action section
for tool in run.required_action.submit_tool_outputs.tool_calls:
  if tool.function.name == "get_current_temperature":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": "57"  ### This is the hard-coded output of what the function should give
    })
  elif tool.function.name == "get_rain_probability":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": "0.06"   ### This is the hard-coded output of what the function should give
    })

### Submit all tool outputs at once after collecting them in a list

In [ ]:
if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    print("Tool outputs submitted successfully.")
  except Exception as e:
    print("Failed to submit tool outputs:", e)
else:
  print("No tool outputs to submit.")
 

In [7]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_ScT2jGZEgWbha7obkWiE25fa', assistant_id='asst_Y50jqDWL3WPuauZzIn40P0Vg', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The current temperature in London is 57°C, and the likelihood of rain today is 6%.'), type='text')], created_at=1713705254, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_0GkOgY8XjT4FH6zxfA33RZ0V', status=None, thread_id='thread_BrLGDifwJvNy6rPmv5W0ihdu'), Message(id='msg_OJLz0jKQ1GJL0Q0LyLtPmc7H', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="What's the weather in London today and the likelihood it'll rain?"), type='text')], created_at=1713705248, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_BrLGDifwJvNy6rPmv5W0ihdu')], object='list', first_id='msg_ScT2jGZEg

# With Streaming

In [4]:
from typing_extensions import override
from openai import AssistantEventHandler
 
class EventHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
      # Retrieve events that are denoted with 'requires_action'
      # since these will have our tool_calls
      if event.event == 'thread.run.requires_action':
        run_id = event.data.id  # Retrieve the run ID from the event data
        self.handle_requires_action(event.data, run_id)
 
    def handle_requires_action(self, data, run_id):
      tool_outputs = []
        
      for tool in data.required_action.submit_tool_outputs.tool_calls:
        if tool.function.name == "get_current_temperature":
          tool_outputs.append({"tool_call_id": tool.id, "output": "57"})
        elif tool.function.name == "get_rain_probability":
          tool_outputs.append({"tool_call_id": tool.id, "output": "0.06"})
        
      # Submit all tool_outputs at the same time
      self.submit_tool_outputs(tool_outputs, run_id)
 
    def submit_tool_outputs(self, tool_outputs, run_id):
      # Use the submit_tool_outputs_stream helper
      with client.beta.threads.runs.submit_tool_outputs_stream(
        thread_id=self.current_run.thread_id,
        run_id=self.current_run.id,
        tool_outputs=tool_outputs,
        event_handler=EventHandler(),
      ) as stream:
        for text in stream.text_deltas:
          print(text, end="", flush=True)
        print()
 
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  event_handler=EventHandler()
) as stream:
  stream.until_done()

The current temperature in London is 57°C, and the likelihood of rain today is 6%.
